# Climate and Soil Data Extraction Script

This script extracts climate and soil data from raster files for specific plots and compiles the data into Excel workbooks. The script uses the arcpy library for geospatial analysis and pandas for data manipulation.

## Prerequisites

1. **ArcGIS Desktop**: The script utilizes arcpy, a component of ArcGIS Desktop. Make sure you have ArcGIS installed and configured.

2. **Python Libraries**: Ensure you have the required Python libraries installed. You can install them using the following command:

   ```bash
   pip install arcpy pandas xlsxwriter
   ```

## Usage

1. **Workspace Setup**: Set the `workspace` variable to the directory containing your raster files.

2. **Plots Information**: Update the `plots` list with the information about your plots. Each entry in the list should contain the plot code, X-coordinate, and Y-coordinate.

3. **Output Excel Files**: Update the `output_excel_file` variable with the desired file path for the output Excel files.

4. **Climate Data Extraction**:

   - Update the `folders` list with the folder names containing the climate data raster files.
   - Ensure that the month names in the `month_names` dictionary match your data's naming convention.

5. **Water Holding Capacity (WHC) Extraction**:

   - Update the `WHC_path` variable with the file path to the WHC raster file.

6. **Run the Notebook Cells**: Execute the script in a Python environment with arcpy, pandas, and xlsxwriter installed. Execute the notebook cells sequentially by clicking on each cell and pressing Shift + Enter. Make sure to run the cells in the correct order.

7. **Output**: The script will create Excel workbooks for climate data and WHC, each containing a worksheet for each plot. The workbooks will be saved at the paths specified in the `output_excel_file` variables.

## Notes

- The script uses regular expressions to extract year and month information from file names. Make sure your file names follow the expected format (e.g., "Output_Switzerland_fst_01_1980").
- The arcpy library is used to perform geospatial operations. Ensure that your arcpy license is valid and configured correctly.
- Remember to uncomment the lines for saving the workbooks (`workbook.save(output_excel_file)`) before running the script in a production environment.

## Disclaimer

This script was created for a specific use case and might need modifications to suit other scenarios. Use it as a reference and adapt it according to your data and requirements.

## Author

Script written by Luca Ferrari
Contact: luca.ferrari@usys.ethz.ch

For any inquiries or assistance, please contact the author.

**Note:** Always back up your data before running scripts that modify or manipulate it.

This README content was generated with the assistance of an AI language model from OpenAI. The provided content is based on user input and has been tailored to the specific requirements of the project.

In [ ]:
import os
import arcpy
from arcpy import env
import pandas as pd
import xlsxwriter
import calendar
import re
arcpy.CheckOutExtension("Spatial")


In [ ]:
# Set the workspace to the directory containing the raster files
workspace = r"N:\Luca_data"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

# Plots 
plots = [
    ['Plot name', x-coord, y-coord],
    ...
]

# Set the output Excel file path
output_excel_file = r"N:\Luca_data\Testpflanzungen\Climate_multisite\Climate_long_term_rest.xls"

# Iterate through the folders
folders = ["Switzerland_tasmax", "Switzerland_tasmin", "Switzerland_pr", "Switzerland_vpd", "Switzerland_rsds", "Switzerland_fst"]

month_names = {
    1: 'Jan',
    2: 'Feb',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
}


In [ ]:
import re

def extract_year(string):
    # Split the string based on underscores
    parts = string.split('_')
    
    # Iterate over the parts and find the first 4-digit year
    for part in parts:
        match = re.match(r"\b\d{4}\b", part)
        if match:
            return int(match.group())
    
    # Return None if no year is found
    return None

def extract_month(string):
    # Split the string based on underscores
    parts = string.split('_')
    
    # Iterate over the parts and find the first 4-digit year
    for part in parts:
        match = re.match(r"\b\d{2}\b", part)
        if match:
            return int(match.group())
    
    # Return None if no year is found
    return None

filename = "Output_Switzerland_fst_01_1980"
year = extract_year(filename)
print("Year:", year)

month = extract_month(filename)
print("Month:", month)

In [ ]:
# Create an Excel workbook and add a worksheet
workbook = xlsxwriter.Workbook(output_excel_file)

for data in plots:  
    # Coordinates of the plot
    x_coord = data[1]  # X-coordinate of the pixel
    y_coord = data[2]  # Y-coordinate of the pixel
    plot_code = data[0]
    # Create the 'Climate_Data' sheet
    worksheet = workbook.add_worksheet(plot_code)

    # Define the header row
    header = ['Year', 'Month', 'Tmax', 'Tmin', 'Rain', 'VPD', 'Solar rad', 'Frost days']
    
    for col, header_name in enumerate(header):
        worksheet.write(0, col, header_name)

    for folder in folders:
        row = 1  # Starting row for data
        
        # Get the list of files in the current folder
        folder_path = os.path.join(workspace, "3PG", folder)
        
        # Get the list of files in the directory using os.scandir()
        with os.scandir(folder_path) as entries:
            # Filter out directories and get only file names
            file_names = [entry.name for entry in entries if entry.is_file()]
            # Filter the list to only include .tif files
            file_names = [f for f in file_names if f.endswith('.flt')]
            file_names = sorted(file_names)


        # Iterate through the files in the folder
        for file in file_names:
            # Extract the year and month from the file name
            filename = os.path.splitext(file)[0]
            year = int(filename.split("_")[0][1:])
            month_num = int(filename.split("_")[2])

            # Extract year and month using regular expressions
            #year = extract_year(filename)
            #month_num = extract_month(filename)

            # Convert the month number into the corresponding month name
            month_name = month_names.get(month_num)

            # Read the raster data as a numpy array
            raster_path = os.path.join(folder_path, file)

            # Get the pixel value at the specified location
            result = arcpy.management.GetCellValue(raster_path, "{} {}".format(x_coord, y_coord))
            pixel_value = float(result.getOutput(0))

            # Write the data to the worksheet
            worksheet.write(row, 0, year)
            worksheet.write(row, 1, month_name)
            worksheet.write(row, folders.index(folder) + 2, pixel_value)  # Convert the value to a float

            row += 1  # Increment the row counter

            print(f"Processed: {file} for plot {plot_code}\n")

# Save the workbook
#workbook.save(output_excel_file)
workbook.close()

In [ ]:
# Extract water holding capacity

# Set the output Excel file path
output_excel_file = r"N:\Luca_data\Testpflanzungen\Climate_multisite\WHC_0_60_test_sites.xls"

# Create an Excel workbook and add a worksheet
workbook = xlsxwriter.Workbook(output_excel_file)
worksheet = workbook.add_worksheet()
# Define the header row
header = ['Plot', 'WHC']

for col, header_name in enumerate(header):
    worksheet.write(0, col, header_name)

row = 1  # Starting row for data

# WHC path
WHC_path = r"N:\Luca_data\HiHydroSoil v2.0\WHC\WCavail_0-60cm_M_250m_Switzerland.tif"

for data in plots:  
    # Coordinates of the plot
    x_coord = data[1]  # X-coordinate of the pixel
    y_coord = data[2]  # Y-coordinate of the pixel
    plot_code = data[0]    

    # Get the pixel value at the specified location
    result = arcpy.management.GetCellValue(WHC_path, "{} {}".format(x_coord, y_coord))
    pixel_value = float(result.getOutput(0))

    # Write the data to the worksheet
    worksheet.write(row, 0, plot_code)
    worksheet.write(row, 1, pixel_value)  # Convert the value to a float

    row += 1  # Increment the row counter

# Save the workbook
#workbook.save(output_excel_file)
workbook.close()

